In [1]:
import re
import os
import pandas as pd


class GprofToCSV: 
    parameters_gprof = (
        'percentageTime',
        'cumulativeTime',
        'selfSeconds',
        'calls',
        'selfMs/call',
        'totalMs/call',
        'function',
        'class'
    )

    def set_file_path(self, path : str):
        self.file_path = path
        self.file_output = f'{self.file_path.split(sep=".txt")[0]}.csv'

    def set_file_output(self, txt : str):
        self.file_output = f'{txt}'

    def convert_file(self):
        dataFile = open(self.file_path) 
        dataOut = open(self.file_output, 'w') 

        pattern = re.compile(r'(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+|\s+)\s+(\d+|\s+)\s+(\d+\.\d+|\s+)\s+(\d+\.\d+|\s+)\s+(.+)')
        
        ptrn_cls_name = re.compile(r'(\w+)[::]')
        ptrn_fnc_name = re.compile(r'[::](\w+)[\(]')

        structBuffer = {}
        for param in self.parameters_gprof:
            structBuffer[param] = []
        print('teste1')
        stop = False
        for line in dataFile:
            line = dataFile.readline()

            stop_condition = re.compile(r'Copying and distribution of this file, with or without modification,')

            if len(stop_condition.findall(line)) > 0:
                stop = True
            
            check = pattern.findall(line)
            if len(check) != 0 and not stop:
                buffer = check[0][:]
                        
                structBuffer['percentageTime'].append(float(buffer[0]))
                structBuffer['cumulativeTime'].append(float(buffer[1]))
                try:
                    structBuffer['selfSeconds'].append(float(buffer[2]))
                except:
                    structBuffer['selfSeconds'].append(-1)

                try:
                    structBuffer['calls'].append(int(buffer[3]))
                except:
                    structBuffer['calls'].append(-1)
                
                try:
                    structBuffer['selfMs/call'].append(float(buffer[4]))
                except:
                    structBuffer['selfMs/call'].append(-1.00)

                try:
                    structBuffer['totalMs/call'].append(float(buffer[5]))
                except:
                    structBuffer['totalMs/call'].append(-1.00)

                className = ptrn_cls_name.findall(buffer[6])
                funcName = ptrn_fnc_name.findall(buffer[6])
                
                if len(className) > 0: 
                    structBuffer['class'].append(className[0])
                    if len(funcName) != 0: 
                        structBuffer['function'].append(funcName[0])
                    else:
                        structBuffer['function'].append(re.compile(r'([\w|\s]+)').findall(buffer[6])[0])
                else:
                    structBuffer['class'].append('void_class')
                    if len(funcName) != 0: 
                        structBuffer['function'].append(funcName[0])
                    else:
                        structBuffer['function'].append(re.compile(r'([\w|\s]+)').findall(buffer[6])[0])
        data_frame = pd.DataFrame(structBuffer).to_csv()
        dataOut.write(data_frame)
        dataFile.close()
        dataOut.close()

class GprofOutCSVReader:
    parameters_gprof = (
        'percentageTime',
        'cumulativeTime',
        'selfSeconds',
        'calls',
        'selfMs/call',
        'totalMs/call',
        'function',
        'class'
    )
    
    def functions_dict(self):
        # turn the csv data by functions into a dict
        self.data_frame = pd.read_csv(self.file_path)        

    def set_file_path(self, file : str = 'akiyo.csv'):
        # check if the file is valid 
        if os.path.isfile(file):
            if len(re.compile(r'\.csv$').findall(file)) > 0:
                self.file_path = file
            else:
                print('Error - file is not a csv file')
        else:
            print('Error - file not found')

    # auxiliar function to find the first occurrency of a name
    
    def __return_element_id__(self, element, d_list : list, key_word : str) -> int:
        for i, j in enumerate(d_list):
            if element == j[key_word]:
                
                return i
            
        return -1

    def split_by_function(self):
        __buffer__ = self.data_frame.to_dict();
        __list_data__ = []
        __dict_buffer__ = {}

        for element in range(0, len(__buffer__['Unnamed: 0'])):
            for i, param in enumerate(__buffer__.keys()):
                __dict_buffer__[param] = __buffer__[param][element]
            __list_data__.append(__dict_buffer__.copy())

        
        
        
        self.list_data_by_class = []
        self.list_data_by_funct = __list_data__[:]


        

        for i, element in enumerate(__list_data__):
            #print(element)
            index = self.__return_element_id__(element['class'], self.list_data_by_class, 'class')
            # index == -1 if the class is not in the list yet
            
            if index == -1:
                self.list_data_by_class.append(element)
            else:
                for parameter in self.parameters_gprof:
                    if type(self.list_data_by_class[index][parameter]) != str:
                        #if self.list_data_by_class[index]['class'] == 'AlfCovariance':
                        #print(self.list_data_by_class[index][parameter])
                        self.list_data_by_class[index][parameter] += element[parameter]
                        #print(self.list_data_by_class[index]['class'])
                        #print()
        
        print('*' *80)
        for i, l in enumerate(self.list_data_by_class):
            print(l['percentageTime'])
            print(l['class'])
            print()
        print('*' *80)
        

        self.dict_data_by_class = {}

        for parameter in self.parameters_gprof:
            self.dict_data_by_class[parameter] = []

        for element in self.list_data_by_class:
            for parameter in self.parameters_gprof:
                self.dict_data_by_class[parameter].append(element[parameter])

        self.dict_data_by_funct = {}

        for parameter in self.parameters_gprof:
            self.dict_data_by_funct[parameter] = []

        for element in self.list_data_by_funct:
            for parameter in self.parameters_gprof:
                self.dict_data_by_funct[parameter].append(element[parameter])


gp1 = GprofOutCSVReader()

gp_to_csv = GprofToCSV()
gp_to_csv.set_file_path('akiyo.txt')
gp_to_csv.set_file_output('akiyo.csv')
gp_to_csv.convert_file()

gp1.set_file_path()
gp1.functions_dict()
gp1.split_by_function()


teste1
********************************************************************************
33.080000000000005
AlfCovariance

11.790000000000001
IntraPrediction

15.620000000000001
DQIntern

11.459999999999997
RdCost

2.7800000000000002
InterpolationFilter

21.03
InterPrediction

6.049999999999999
CodingStructure

15.689999999999987
void_class

0.78
QuantRDOQ

109.67999999999999
EncCu

11.340000000000002
IntraSearch

10.030000000000001
TrQuant

3.7
Picture

2.6900000000000004
MatrixIntraPrediction

1.52
PelStorage

7.140000000000001
InterSearch

1.9599999999999997
CoeffCodingContext

0.37
QpParam

7.0299999999999985
CABACWriter

0.36
CodingUnit

0.71
TU

0.39
QTBTPartitioner

0.6100000000000001
EncTemporalFilter

1.08
DepQuant

0.82
std

2.33
PU

0.49
CS

0.29000000000000004
BitEstimatorBase

0.54
CU

0.13999999999999999
BestEncInfoCache

35.8
EncAdaptiveLoopFilter

0.17
MergeCtx

0.2
EncModeCtrlMTnoRQT

0.25
DeriveCtx

0.04
IbcHashMap

0.06
CompArea

0.16
UnitArea

0.06
EncModeCtrl

0.04


In [54]:
import re

data_path = 'akiyo.txt'

pattern = re.compile(r'(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+)?\s+(\d+\.\d+)?\s+(\d+\.\d+)?\s+(.+?)(?::){2}?(.+)(?:\(){1}?')

f = open(data_path)

for line in f:
    print(pattern.findall(line))



[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
